<a href="https://colab.research.google.com/github/Danusha2004/AI_Text_Summarize/blob/main/Text_Summary_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate sentence-transformers spacy nltk networkx streamlit matplotlib
!python -m spacy download en_core_web_sm

In [ ]:
!python -m nltk.downloader punkt stopwords

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


# Preprocessing Functions
Text cleaning, tokenization, and preparation for summarization models.

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import re
from nltk.tokenize import sent_tokenize

def clean_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def split_sentences(text):
    text = clean_text(text)
    return sent_tokenize(text)

# Test
sample = "Hello! This is a test. Let's see if it works."
split_sentences(sample)


# 3. Extractive Summarization - TextRank
Using TextRank algorithm to select the most important sentences.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx

def textrank_summary(text, n_sentences=3):
    sentences = sent_tokenize(text)
    if len(sentences) <= n_sentences:
        return " ".join(sentences)

    tfidf = TfidfVectorizer().fit_transform(sentences)
    sim_matrix = (tfidf * tfidf.T).toarray()

    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank_numpy(nx_graph)

    ranked = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    top_sentences = [s for _, s in ranked[:n_sentences]]

    return " ".join(top_sentences)

# Test
text = """Natural language processing (NLP) is a subfield of AI that deals with
the interaction between computers and humans using natural language."""
print(textrank_summary(text, 2))


# 4. Abstractive Summarization
Using pre-trained transformer models like BART and T5 to generate summaries.


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """The history of NLP dates back to the 1950s, when computer scientists first began exploring
how machines could process and understand human language. The 1950s marked the dawn of modern natural language processing (NLP) as a formal field,
emerging from broader efforts in artificial intelligence and computational linguistics.
Several pioneering computer scientists contributed foundational ideas and experiments that shaped the trajectory of NLP.
Natural Language Processing (NLP) is a field at the intersection of linguistics and computer science that focuses on enabling machines to understand,
interpret, and generate human language. It leverages machine learning (ML) to learn patterns from large text corpora,
enabling tasks like sentiment analysis, named entity recognition, and machine translation.
Deep learning (DL), a subset of ML, uses neural networks with many layers to model complex language representations,
driving state-of-the-art performance in areas such as language modeling, question answering, and conversational agents.
Together, NLP, ML, and DL empower systems to parse syntax, capture semantics, and produce fluent, contextually relevant language,
while continually benefiting from advances in data, computation, and algorithms."""
summary = summarizer(text, max_length=60, min_length=20, do_sample=False)

print("Generated Summary:", summary[0]['summary_text'])


# 6. Evaluation with ROUGE
Evaluate the quality of generated summaries using ROUGE metrics.


In [ ]:
!pip install evaluate


In [ ]:
!pip install rouge_score

In [ ]:
import evaluate

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Human-written reference (original text)
references = ["""The history of NLP dates back to the 1950s, when computer scientists first began exploring
how machines could process and understand human language. The 1950s marked the dawn of modern natural language processing (NLP) as a formal field,
emerging from broader efforts in artificial intelligence and computational linguistics.
Several pioneering computer scientists contributed foundational ideas and experiments that shaped the trajectory of NLP.
Natural Language Processing (NLP) is a field at the intersection of linguistics and computer science that focuses on enabling machines to understand,
interpret, and generate human language. It leverages machine learning (ML) to learn patterns from large text corpora,
enabling tasks like sentiment analysis, named entity recognition, and machine translation.
Deep learning (DL), a subset of ML, uses neural networks with many layers to model complex language representations,
driving state-of-the-art performance in areas such as language modeling, question answering, and conversational agents.
Together, NLP, ML, and DL empower systems to parse syntax, capture semantics, and produce fluent, contextually relevant language,
while continually benefiting from advances in data, computation, and algorithms."""]

# Model-generated summary
predictions = ["The 1950s marked the dawn of modern natural language processing (NLP) as a formal field. "
               "NLP leverages machine learning (ML) to learn patterns from large text corpora. "
               "Deep learning (DL), a subset of ML, uses neural networks with many layers to model complex."]

# Compute ROUGE scores
results = rouge.compute(predictions=predictions, references=references)

print(results)



# 7. Visualization
Plot ROUGE scores for different summarization models.


In [ ]:
import matplotlib.pyplot as plt

scores = {'rouge1': 0.85, 'rouge2': 0.67, 'rougeL': 0.80}
plt.bar(scores.keys(), scores.values())
plt.title("ROUGE Evaluation Scores")
plt.ylabel("Score")
plt.show()


# 5. Fine-Tuning the Abstractive Model
Train BART or T5 on a custom dataset or a public dataset like CNN/DailyMail.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)


dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)



In [ ]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

import numpy as np
import nltk
import os

nltk.download("punkt")

In [ ]:
# -------- Configuration --------
model_name = "facebook/bart-base"  # or "t5-base"
dataset_name = "cnn_dailymail"
dataset_config = "3.0.0"
text_column = "article"     # source column for CNN/DailyMail
summary_column = "highlights"  # target column (reference summary)
max_input_length = 256
max_target_length = 64
batch_size = 2
num_train_epochs = 1
output_dir = "./bart_base_summarization_fast"
seed = 42
# -------------------------------

In [ ]:
# Load dataset
dataset = load_dataset(dataset_name, dataset_config)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Preprocessing
def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[summary_column]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length"  # ensure fixed size to speed up data collator on small runs
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100)


In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
# =========================
# 1. Install Required Packages
# =========================
!pip install transformers datasets evaluate nltk

# =========================
# 2. Import Libraries
# =========================
import numpy as np
import nltk
import evaluate
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

# Download punkt for sentence tokenization
nltk.download("punkt")

# =========================
# 3. Load Dataset
# =========================
dataset = load_dataset("cnn_dailymail", "3.0.0")

# =========================
# 4. Load Tokenizer and Model
# =========================
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# =========================
# 5. Preprocessing Function
# =========================
text_column = "article"
summary_column = "highlights"
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    # Tokenize inputs
    model_inputs = tokenizer(
        examples[text_column],
        max_length=max_input_length,
        truncation=True
    )

    # Tokenize labels (summaries)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples[summary_column],
            max_length=max_target_length,
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# =========================
# 6. Tokenize Dataset
# =========================
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# =========================
# 7. Small Subset for Quick Training
# =========================
small_train_dataset = tokenized_datasets["train"].select(range(1000))
small_val_dataset = tokenized_datasets["validation"].select(range(200))

# =========================
# 8. Data Collator
# =========================
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# =========================
# 9. Define ROUGE Evaluation Metric
# =========================
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    preds = ["\n".join(nltk.sent_tokenize(p)) for p in preds]
    labels = ["\n".join(nltk.sent_tokenize(l)) for l in labels]
    return preds, labels

def compute_metrics(eval_pred):
    generated_ids, label_ids = eval_pred
    preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    preds, labels = postprocess_text(preds, labels)
    result = rouge.compute(predictions=preds, references=labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}

    # Track average generated length
    pred_lens = [np.count_nonzero(g != tokenizer.pad_token_id) for g in generated_ids]
    result["gen_len"] = float(np.mean(pred_lens))
    return result

# =========================
# 10. Training Configuration
# =========================
output_dir = "./bart_summarizer"
batch_size = 2
seed = 42

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=2,
    num_train_epochs=1,          # keep it low for testing
    fp16=torch.cuda.is_available(),
    learning_rate=3e-5,
    weight_decay=0.01,
    dataloader_num_workers=2,
    seed=seed,
    remove_unused_columns=True,
    push_to_hub=False,
)

# =========================
# 11. Initialize Trainer
# =========================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# =========================
# 12. Train Model
# =========================
trainer.train()

# =========================
# 13. Save Model & Tokenizer
# =========================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# =========================
# 14. Evaluate Model
# =========================
metrics = trainer.evaluate(eval_dataset=small_val_dataset)
print("Evaluation Metrics:", metrics)

# =========================
# 15. Generate Summaries on Test Data
# =========================
test_samples = dataset["test"].select(range(2))
inputs = [ex[text_column] for ex in test_samples]

inputs_tokenized = tokenizer(
    inputs,
    max_length=512,
    truncation=True,
    return_tensors="pt",
    padding=True
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

input_ids = inputs_tokenized["input_ids"].to(device)
attention_mask = inputs_tokenized["attention_mask"].to(device)

generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=128,
    num_beams=2,
    early_stopping=True
)

generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

# =========================
# 16. Display Results
# =========================
for i, gen in enumerate(generated_texts):
    print(f"\n=== Example {i+1} ===")
    print("Article snippet:", inputs[i][:500].replace("\n", " "))
    print("Generated summary:", gen)
    print("Reference summary:", test_samples[i][summary_column])


In [ ]:
import nltk

# Download punkt and punkt_tab
nltk.download('punkt')
nltk.download('punkt_tab')
